In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

# Cargar el raster
raster_path = 'spatial_data/badajoz_multiclass_mask.tif'
raster = rasterio.open(raster_path)

# Leer los datos del raster
raster_data = raster.read(1)

# Definir los colores para cada clase, con verde oscuro
class_colors = {
    0: (0, 0, 0),       # Negro para 'null'
    1: (204, 204, 0),     # Verde oscuro para 'Invernaderos'
    2: (255, 0, 0),     # Rojo para 'Solar'
    3: (173, 216, 230)  # Azul claro con tono grisáceo para 'Otros'
}

# Normalizar los colores a 0-1 para matplotlib
class_colors_normalized = {key: np.array(color)/255.0 for key, color in class_colors.items()}

# Crear una lista de colores en el formato adecuado para matplotlib
cmap = mcolors.ListedColormap([class_colors_normalized[key] for key in sorted(class_colors_normalized.keys())])

# Crear una lista de límites de clase, con un valor extra para el límite superior de la última clase
bounds = np.arange(len(class_colors) + 1) - 0.5
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# Visualizar el raster con el colormap personalizado
plt.figure(figsize=(10, 10))
plt.imshow(raster_data, cmap=cmap, norm=norm)
cbar = plt.colorbar(ticks=[0, 1])  # Añadir colorbar con las etiquetas de las clases
cbar.set_ticklabels(['Otros', 'Solar'])  # Etiquetas de colorbar

# Añadir una leyenda personalizada
legend_patches = [mpatches.Patch(color=class_colors_normalized[key], label=f"{key}: {'null' if key == 0 else 'Invernaderos' if key == 1 else 'Solar' if key == 2 else 'Otros'}") for key in sorted(class_colors_normalized.keys())]
plt.legend(handles=legend_patches, loc='lower right', title='Clases')

plt.title('Máscara multiclase Badajoz, 2019')
plt.show()


In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Cargar el raster
raster_path = 'spatial_data/badajoz_multiclass_mask.tif'
raster = rasterio.open(raster_path)

# Leer los datos del raster
raster_data = raster.read(1)

# Contar los píxeles para cada valor, excluyendo los 0
unique, counts = np.unique(raster_data, return_counts=True)
pixel_counts = dict(zip(unique, counts))

# Crear un gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(pixel_counts.keys(), pixel_counts.values(), color=['#ADD8E6', '#FF0000'])  # Usar los colores definidos anteriormente
plt.xlabel('Clase')
plt.ylabel('Cantidad de píxeles')
plt.title('Píxeles por clase (sin 0), original')
plt.xticks(ticks=[0, 1], labels=['Otros', 'Solar'])  # Etiquetas de las clases

# Añadir etiquetas con los valores numéricos encima de las barras
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{height}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),  # Desplazamiento del texto en píxeles
                 textcoords="offset points",
                 ha='center', va='bottom')

plt.show()


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Ruta a las imágenes PNG
images_path = 'dataset/clean/labels'

# Inicializar un contador para los píxeles
pixel_counter = Counter()

# Leer todas las imágenes PNG en la ruta especificada
for filename in os.listdir(images_path):
    if filename.endswith('.png'):
        image_path = os.path.join(images_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Leer la imagen en modo de escala de grises
        unique, counts = np.unique(image, return_counts=True)
        pixel_counter.update(dict(zip(unique, counts)))

# Eliminar los píxeles con valor 0
if 0 in pixel_counter:
    del pixel_counter[0]

# Colores asociados a cada clase
colors = ['#8CFF32', '#ADD8E6', '#FF0000']  # Amarillo, rojo, azul

# Crear un gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(pixel_counter.keys(), pixel_counter.values(), color=colors)  # Usar los colores definidos
plt.xlabel('Clase')
plt.ylabel('Cantidad de píxeles')
plt.title('Píxeles por clase (sin 0), filtrado')
plt.xticks(ticks=[1, 2, 3], labels=['Invernaderos', 'Solar', 'Otros'])  # Etiquetas de las clases

# Añadir etiquetas con los valores numéricos encima de las barras
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{height}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),  # Desplazamiento del texto en píxeles
                 textcoords="offset points",
                 ha='center', va='bottom')

plt.show()



In [ ]:
import os
import numpy as np
from PIL import Image
from collections import defaultdict

# Ruta de la carpeta que contiene las imágenes
carpeta_imagenes = 'dataset/clean/labels'

# Diccionario para almacenar el conteo de píxeles por clase
conteo_pixeles = defaultdict(lambda: {1: 0, 2: 0, 3: 0})

# Listas para almacenar las imágenes con más píxeles para cada clase
imagenes_mas_pixeles = {1: [], 2: [], 3: []}

# Recorremos todos los archivos en la carpeta de imágenes
for archivo in os.listdir(carpeta_imagenes):
    if archivo.endswith('.png'):
        # Abrimos la imagen
        imagen_path = os.path.join(carpeta_imagenes, archivo)
        imagen = Image.open(imagen_path)
        # Convertimos la imagen a un array de numpy
        array_imagen = np.array(imagen)
        
        # Contamos los píxeles de cada clase (1, 2, 3)
        conteo_clase_1 = np.sum(array_imagen == 1)
        conteo_clase_2 = np.sum(array_imagen == 2)
        conteo_clase_3 = np.sum(array_imagen == 3)
        
        # Guardamos el conteo de píxeles en el diccionario
        conteo_pixeles[archivo] = {1: conteo_clase_1, 2: conteo_clase_2, 3: conteo_clase_3}

# Encontramos las 3 imágenes con más píxeles para cada clase
for clase in [1, 2, 3]:
    # Ordenamos las imágenes por el número de píxeles para la clase actual en orden descendente
    ordenado_por_clase = sorted(conteo_pixeles.items(), key=lambda x: x[1][clase], reverse=True)
    # Guardamos las 3 imágenes con más píxeles para la clase actual
    imagenes_mas_pixeles[clase] = [item[0] for item in ordenado_por_clase[:3]]

# Imprimimos los resultados
for clase in [1, 2, 3]:
    print(f'Imágenes con más píxeles para la clase {clase}:')
    for imagen in imagenes_mas_pixeles[clase]:
        print(f'  {imagen} con {conteo_pixeles[imagen][clase]} píxeles')
    print()


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Definir la ruta de la carpeta que contiene las imágenes de características
carpeta_features = 'dataset/clean/features'

# Definir el objeto de imágenes con más píxeles por clase
imagenes_mas_pixeles = {
    'Invernaderos': [
        'square_12800_4608.png',
        'square_4608_6656.png',
        'square_12288_5120.png'
    ],
    'Solar': [
        'square_4096_7680.png',
        'square_14848_3072.png',
        'square_9216_11776.png'
    ],
    'Otros': [
        'square_14848_8192.png',
        'square_5632_6144.png',
        'square_5632_8192.png'
    ]
}

# Buscar archivos en la carpeta de características que contengan alguno de los sufijos
def buscar_archivos_con_sufijos(carpeta_features, sufijos):
    archivos_encontrados = []
    for archivo in os.listdir(carpeta_features):
        if any(sufijo in archivo for sufijo in sufijos):
            archivos_encontrados.append(os.path.join(carpeta_features, archivo))
    return archivos_encontrados

# Obtener los archivos que coinciden con los sufijos para cada clase
archivos_por_clase = {}
for clase, imagenes in imagenes_mas_pixeles.items():
    archivos_por_clase[clase] = buscar_archivos_con_sufijos(carpeta_features, imagenes)

# Imprimir los archivos encontrados
for clase, archivos in archivos_por_clase.items():
    print(f'\nArchivos encontrados para la clase {clase}:')
    for archivo in archivos:
        print(archivo)

# Función para mostrar imágenes en una cuadrícula por clase
def mostrar_imagenes_por_clase(archivos_por_clase, filas=3, columnas=3):
    # Crear la figura con una cuadrícula
    fig, axs = plt.subplots(filas, columnas, figsize=(12, 12))
    
    # Lista de clases ordenadas por columna
    clases = list(archivos_por_clase.keys())
    
    for i, clase in enumerate(clases):
        for j, archivo in enumerate(archivos_por_clase[clase]):
            if j >= filas:
                break  # Si hay más imágenes que el número de filas, solo tomamos las primeras
            
            # Obtener la imagen
            imagen = Image.open(archivo)
            # Ubicación en la cuadrícula
            axs[j, i].imshow(imagen, cmap='gray')
            axs[j, i].set_title(clase)
            axs[j, i].axis('off')

    # Configurar títulos de columnas
    for i, clase in enumerate(clases):
        axs[-1, i].set_xlabel(clase, fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Mostrar imágenes en una cuadrícula por clase
mostrar_imagenes_por_clase(archivos_por_clase)
